# Implementing Decision Tree Classifier


## Importing Libraries


In [76]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from copy import deepcopy
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from scipy.sparse import coo_matrix


## Importing and preprocessing the data


In [77]:
df = pd.read_csv(r'./data/nursery.csv')
df.dropna(inplace=True)


In [78]:
df.count()


parents             12960
has_nurs            12960
form                12960
children            12960
housing             12960
finance             12960
social              12960
health              12960
final evaluation    12960
dtype: int64

There is no missing data in the dataset. So, we can directly move on to the next step.


In [79]:
y = df['final evaluation']
X = df.drop(labels=['final evaluation'], axis=1)


## Implementing the model


In [94]:
class DecisionTreeClassifier:
    def __init__(self, max_depth=None, impurity='entropy'):
        self.max_depth = max_depth
        self.main_tree = None
        self.leaf_name = 'leaf'
        self.number_of_nodes = None
        self.number_of_leafs = None
        self.label_encoder = None
        self.X_encoder = None
        self.train_errors_list = None
        self.test_errors_list = None
        self.validation_errors_list = None
        self.list_of_nodes = None
        self.impurity_name = impurity

        if max_depth is not None and self.max_depth <= 0:
            raise Exception('max_depth should be positive')

        if impurity == 'entropy':
            self.impurity = self._get_entropy

        elif impurity == 'gini':
            self.impurity = self._get_gini

    def encode_X(self, X):
        X = np.array(X)
        if self.X_encoder is None:
            self.X_encoder = [LabelEncoder().fit(X[:, i])
                              for i in range(X.shape[1])]

        return np.concatenate([
            self.X_encoder[i].transform(X[:, i]).reshape(-1, 1)
            for i in range(X.shape[1])], axis=1)

    def encode_y(self, y):
        self.label_encoder = LabelEncoder()
        return self.label_encoder.fit_transform(y)

    def decode_y(self, y):

        if self.label_encoder is None:
            raise Exception('Label encoder is not initialized')

        return self.label_encoder.inverse_transform(y)

    def _get_probablity(self, X):
        return np.bincount(X)/X.shape[0]

    def _get_entropy(self, X):
        X = self._get_probablity(X)
        X = X[X != 0]
        return np.dot(X, -np.log2(X))

    def _get_gini(self, X):
        X = self._get_probablity(X)
        return 1 - np.dot(X, X)

    def check_is_fitted(self):
        if self.main_tree is None:
            raise Exception('model is not trained')

    def check_is_fit_prune_evaluate(self):
        if self.train_errors_list is None:
            raise Exception('fit_prune_evaluate is not called')

    def get_information_gain(self, X, y):

        if len(X.shape) != 2:
            raise Exception('X should be 2D array')

        info_gain = list()
        for i in range(X.shape[1]):
            X_selected = X[:, i]

            info_gain.append(self.impurity(y)
                             - np.sum(
                                 [self._get_probablity(X_selected)[j] *
                                  self.impurity(y[X_selected == j])
                                  for j in np.unique(X_selected)]))

        return np.array(info_gain)

    def _name_branch(self, depth, branch_number):
        return branch_number

    def _split_nodes(self, X, y):

        if len(X.shape) != 2:
            raise Exception('X should be 2D array')

        if len(self.count_categories(X)) == 0:
            raise Exception('model is not trained')

        self.number_of_nodes += 1
        X_feature_arg = np.argmax(self.get_information_gain(X[1:, :], y))
        feature_arg = X[0, X_feature_arg]
        leftover_features = np.delete(X[0, :], X_feature_arg).reshape(1, -1)
        X = X[1:, :]
        X_selected = X[:, X_feature_arg]
        X = np.delete(X, X_feature_arg, axis=1)

        sub_nodes = dict()

        for i in self.columns_categories[feature_arg]:

            splitted_data = np.concatenate((leftover_features,
                                            X[X_selected == i]), axis=0)

            splitted_labels = y[X_selected == i]

            branch_name = self._name_branch(self._depth_iter, i)

            if len(splitted_labels) == 0:
                sub_nodes[branch_name] = {
                    self.leaf_name: [np.bincount(y).argmax(), [len(y)]]}
                self.number_of_leafs += 1
                continue

            if np.all(splitted_labels == splitted_labels[0]):
                sub_nodes[branch_name] = {self.leaf_name: [
                    splitted_labels[0], [len(splitted_labels)]]}
                self.number_of_leafs += 1
                continue

            sub_nodes[branch_name] = (splitted_data, splitted_labels)

        return {(feature_arg, tuple(np.bincount(y))): sub_nodes}

    def count_categories(self, columns):
        if len(columns.shape) != 2:
            raise Exception('X should be 2D array')

        return [np.unique(columns[:, i]) for i in range(columns.shape[1])]

    def summary(self):
        self.check_is_fitted()
        print('Impurity: {}'.format(self.impurity_name))
        print('Number of nodes: {}'.format(self.number_of_nodes))
        print('Number of leafs: {}'.format(self.number_of_leafs))
        print('Max depth: {}'.format(self.max_depth))

    def fit(self, X, y):

        if len(X.shape) != 2:
            raise Exception('X should be 2D array')

        if len(y.shape) != 1:
            raise Exception('y should be 1D array')

        if X.shape[0] != y.shape[0]:
            raise Exception('X and y should have same number of rows')

        if self.max_depth is None:
            self.max_depth = X.shape[1]

        self.columns_categories = self.count_categories(X)
        self._depth_iter = 0
        self.number_of_nodes = 0
        self.number_of_leafs = 0

        X = np.concatenate((np.arange(X.shape[1]).reshape(1, -1), X), axis=0)

        self.main_tree = self._split_nodes(X, y)
        sub_trees = [self.main_tree]

        for self._depth_iter in range(1, self.max_depth):
            sub_trees_temp = []
            for sub_tree in sub_trees:
                node = list(sub_tree.keys())[0]
                for branch in sub_tree[node]:

                    X_ys = sub_tree[node][branch]
                    if type(X_ys) == tuple:
                        sub_tree[node][branch] = self._split_nodes(
                            X_ys[0], X_ys[1])
                        sub_trees_temp.append(sub_tree[node][branch])

                sub_trees = sub_trees_temp

        # Replacing the not learned data with the maximum frequent one
        for sub_tree in sub_trees:
            node = list(sub_tree.keys())[0]
            for branch in sub_tree[node]:
                X_ys = sub_tree[node][branch]
                if type(X_ys) == tuple:
                    sub_tree[node][branch] = {self.leaf_name: [np.bincount(
                        X_ys[1]).argmax(), np.bincount(X_ys[1]).tolist()]}
                    self.number_of_leafs += 1
                    continue

        self.number_of_nodes += self.number_of_leafs

        return self.main_tree

    def _predict_on_tree(self, X, tree):

        self.check_is_fitted()
        if len(X.shape) != 2:
            raise Exception('X should be 2D array')

        y_predicted = list()
        for x in X:
            sub_tree = tree
            for depth in range(self.get_depth(tree)):
                node = list(sub_tree.keys())[0]
                if node == self.leaf_name:
                    break
                x_class = self._name_branch(depth, x[node[0]])
                sub_tree = sub_tree[node][x_class]

            y_predicted.append(sub_tree[self.leaf_name][0])

        return np.array(y_predicted)

    def _score_on_tree(self, X, y, tree):

        return np.sum(self._predict_on_tree(X, tree) == y)/y.shape[0]

    def _error_on_tree(self, X, y, tree):

        return 1 - self._score_on_tree(X, y, tree)

    def predict(self, X):
        return self._predict_on_tree(X, self.main_tree)

    def score(self, X, y):
        return self._score_on_tree(X, y, self.main_tree)

    def error(self, X, y):
        return self._error_on_tree(X, y, self.main_tree)

    def get_depth(self, tree):
        sub_trees = [tree]
        depth = 0
        while True:
            sub_trees_temp = []
            for sub_tree in sub_trees:
                node = list(sub_tree.keys())[0]

                for branch in sub_tree[node]:

                    sub_branch = sub_tree[node][branch]
                    if list(sub_branch.keys())[0] != self.leaf_name:
                        sub_trees_temp.append(sub_branch)

            depth += 1
            sub_trees = sub_trees_temp
            if len(sub_trees) == 0:
                return depth

    def _count_nodes(self, tree, depth):
        if len(tree) != 1:
            raise Exception('tree should have one root node')

        number_of_nodes = 1
        sub_trees = [tree]
        for _ in range(depth):
            sub_trees_temp = []
            for sub_tree in sub_trees:
                node = list(sub_tree.keys())[0]

                for branch in sub_tree[node]:
                    number_of_nodes += 1
                    sub_branch = sub_tree[node][branch]
                    if list(sub_branch.keys())[0] != self.leaf_name:
                        sub_trees_temp.append(sub_branch)

            sub_trees = sub_trees_temp

        return number_of_nodes

    def _count_leafs(self, tree, depth):
        if len(tree) != 1:
            raise Exception('tree should have one root node')

        number_of_leafs = 0
        sub_trees = [tree]
        for _ in range(depth):
            sub_trees_temp = []
            for sub_tree in sub_trees:
                node = list(sub_tree.keys())[0]

                for branch in sub_tree[node]:
                    sub_branch = sub_tree[node][branch]
                    if list(sub_branch.keys())[0] != self.leaf_name:
                        sub_trees_temp.append(sub_branch)
                    else:
                        number_of_leafs += 1

            sub_trees = sub_trees_temp

        return number_of_leafs

    def count_nodes(self, tree):
        return self._count_nodes(tree, self.get_depth(tree))

    def count_leafs(self, tree):
        return self._count_leafs(tree, self.get_depth(tree))

    def _get_subtrees(self, tree):

        if type(tree) != dict:
            raise Exception('tree should be dict')

        sub_trees = [[tree]]
        for depth in range(1, self.get_depth(tree)):
            sub_trees_on_depth = []
            for sub_tree in sub_trees[depth-1]:
                node = list(sub_tree.keys())[0]
                for branch in sub_tree[node]:
                    sub_branch = sub_tree[node][branch]
                    if list(sub_branch.keys())[0] != self.leaf_name:
                        sub_trees_on_depth.append(sub_branch)

            sub_trees.append(sub_trees_on_depth)

        return sub_trees

    def _fit_prune_evaluate(self, X_val, y_val,
                            X_train=None,
                            y_train=None,
                            X_test=None,
                            y_test=None,
                            max_prune_depth=1):

        self.check_is_fitted()

        if len(X_val.shape) != 2:
            raise Exception('X should be 2D array')
        if len(y_val.shape) != 1:
            raise Exception('y should be 1D array')
        if X_val.shape[0] != y_val.shape[0]:
            raise Exception('X and y should have the same number of rows')
        if max_prune_depth >= self.max_depth:
            raise Exception('max_prune_depth should be less than max_depth')

        if (X_train is not None):
            self.fit(X_train, y_train)
            self.train_errors_list = [self.score(X_train, y_train)]
            self.test_errors_list = [self.score(X_test, y_test)]
            self.validation_errors_list = [self.score(X_val, y_val)]
            self.list_of_nodes = [self.number_of_nodes]

        self.main_tree_pruned = deepcopy(self.main_tree)

        sub_trees = self._get_subtrees(self.main_tree_pruned)

        sampled_validation_set_number = int(np.log2(len(X_val)))

        for depth in range(self.max_depth-1,
                           self.max_depth-max_prune_depth-1,
                           -1):

            for sub_tree in sub_trees[depth]:
                node = list(sub_tree.keys())[0]
                node_samples = np.array(node[1])
                if node is self.leaf_name:
                    continue

                back_up_sub_tree = sub_tree[node]
                sub_tree.pop(node)

                sub_tree[self.leaf_name] = [np.argmax(node_samples),
                                            node_samples.tolist()]

                # Shuffling the validation set
                X_val_sparse = coo_matrix(X_val)
                X_val, X_sparse, y_val = shuffle(
                    X_val, X_val_sparse, y_val, random_state=None)

                # Compare the validation error with the previous one
                pruned_tree_error = self._error_on_tree(
                    X_val[:sampled_validation_set_number],
                    y_val[:sampled_validation_set_number],
                    self.main_tree_pruned)

                main_tree_error = self._error_on_tree(
                    X_val[:sampled_validation_set_number],
                    y_val[:sampled_validation_set_number],
                    self.main_tree)

                if pruned_tree_error <= main_tree_error:
                    print('hello')
                    self.main_tree = deepcopy(self.main_tree_pruned)

                    self.max_depth = self.get_depth(self.main_tree)
                    self.number_of_leafs = self._count_leafs(
                        self.main_tree, self.max_depth)
                    self.number_of_nodes = self._count_nodes(
                        self.main_tree, self.max_depth)

                    if (X_train is not None):
                        self.train_errors_list.append(self.error(X_train,
                                                                 y_train))

                        self.test_errors_list.append(self.error(X_test,
                                                                y_test))
                        self.validation_errors_list.append(self.error(X_val,
                                                                      y_val))
                        self.list_of_nodes.append(self.number_of_nodes)

                else:
                    sub_tree[node] = back_up_sub_tree

    def prune(self, X_val, y_val, max_prune_depth=1):
        return self._fit_prune_evaluate(X_val=X_val,
                                        y_val=y_val,
                                        max_prune_depth=max_prune_depth)

    def fit_prune_evaluate(self,
                           X_train,
                           y_train,
                           X_val,
                           y_val,
                           X_test,
                           y_test,
                           max_prune_depth=1):
        return self._fit_prune_evaluat(X_val=X_val,
                                       y_val=y_val,
                                       X_train=X_train,
                                       y_train=y_train,
                                       X_test=X_test,
                                       y_test=y_test,
                                       max_prune_depth=max_prune_depth)

    def plot_error_curvs(self, figsize=(10, 10)):
        self.check_is_fit_prune_evaluate()

        plt.figure(figsize=figsize)
        plt.plot(self.number_of_nodes,
                 self.train_errors_list, label='train error')
        plt.plot(self.number_of_nodes,
                 self.test_errors_list, label='test error')
        plt.plot(self.number_of_nodes,
                 self.validation_errors_list, label='validation error')
        plt.legend()
        plt.show()


## Encoding the data

In [81]:
model = DecisionTreeClassifier()
X_encoded = model.encode_X(X)
y_encoded = model.encode_y(y)

##


## Evaluate the the model with different size of training set and different hyperparameters

In [82]:
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y_encoded, test_size=0.5, random_state=42)

model = DecisionTreeClassifier(max_depth=None, impurity='entropy')

model.fit(X_train, y_train)
print('training size 50%')
print('Train score: {:.4}'.format(
    model.score(X_train, y_train)))
print('Test score: {:.4}'.format(model.score(X_test, y_test)))


training size 50%
Train score: 1.0
Test score: 0.9651


In [83]:
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y_encoded, test_size=0.25, random_state=42)

model = DecisionTreeClassifier(max_depth=None, impurity='entropy')
model.fit(X_train, y_train)
model.summary()
print('training size 75%')
print('Train score: {:.4}'.format(
    model.score(X_train, y_train)))
print('Test score: {:.4}'.format(model.score(X_test, y_test)))


Impurity: entropy
Number of nodes: 1075
Number of leafs: 767
Max depth: 8
training size 75%
Train score: 1.0
Test score: 0.9793


In [84]:
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y_encoded, test_size=0.5, random_state=42)

model = DecisionTreeClassifier(max_depth=6, impurity='entropy')
model.fit(X_train, y_train)
model.summary()
print('training size 50%')
print('Train score: {:.4}'.format(
    model.score(X_train, y_train)))
print('Test score: {:.4}'.format(model.score(X_test, y_test)))


Impurity: entropy
Number of nodes: 571
Number of leafs: 405
Max depth: 6
training size 50%
Train score: 0.9784
Test score: 0.9551


In [85]:
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y_encoded, test_size=0.25, random_state=42)

model = DecisionTreeClassifier(max_depth=6, impurity='entropy')
model.fit(X_train, y_train)
model.summary()
print('training size 75%')
print('Train score: {:.4}'.format(
    model.score(X_train, y_train)))
print('Test score: {:.4}'.format(model.score(X_test, y_test)))


Impurity: entropy
Number of nodes: 589
Number of leafs: 416
Max depth: 6
training size 75%
Train score: 0.9754
Test score: 0.966


#### Gini impurity

In [86]:
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y_encoded, test_size=0.5, random_state=42)

model = DecisionTreeClassifier(max_depth=None, impurity='gini')
model.fit(X_train, y_train)
model.summary()
print('training size 50%')
print('Train score: {:.4}'.format(
    model.score(X_train, y_train)))
print('Test score: {:.4}'.format(model.score(X_test, y_test)))


Impurity: gini
Number of nodes: 931
Number of leafs: 663
Max depth: 8
training size 50%
Train score: 1.0
Test score: 0.9681


In [87]:
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y_encoded, test_size=0.25, random_state=42)

model = DecisionTreeClassifier(max_depth=None, impurity='gini')
model.fit(X_train, y_train)
model.summary()
print('training size 75%')
print('Train score: {:.4}'.format(
    model.score(X_train, y_train)))
print('Test score: {:.4}'.format(model.score(X_test, y_test)))


Impurity: gini
Number of nodes: 1062
Number of leafs: 756
Max depth: 8
training size 75%
Train score: 1.0
Test score: 0.9793


In [88]:
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y_encoded, test_size=0.5, random_state=42)

model = DecisionTreeClassifier(max_depth=6, impurity='gini')
model.fit(X_train, y_train)
model.summary()
print('training size 50%')
print('Train score: {:.4}'.format(
    model.score(X_train, y_train)))
print('Test score: {:.4}'.format(model.score(X_test, y_test)))


Impurity: gini
Number of nodes: 578
Number of leafs: 410
Max depth: 6
training size 50%
Train score: 0.9798
Test score: 0.9565


In [89]:
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y_encoded, test_size=0.25, random_state=42)

model = DecisionTreeClassifier(max_depth=6, impurity='gini')
model.fit(X_train, y_train)
model.summary()
print('training size 75%')
print('Train score: {:.4}'.format(
    model.score(X_train, y_train)))
print('Test score: {:.4}'.format(model.score(X_test, y_test)))


Impurity: gini
Number of nodes: 580
Number of leafs: 408
Max depth: 6
training size 75%
Train score: 0.9756
Test score: 0.966


## Pruning the tree

In [100]:

X_train_val, X_test, y_train_val, y_test = train_test_split(
    X_encoded, y_encoded, test_size=0.1, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=2/9, random_state=42)

model = DecisionTreeClassifier(max_depth=None, impurity='gini')

model.fit(X_train, y_train)


{(7, (3050, 2989, 2, 2808, 223)): {0: {'leaf': [0, [3050]]},
  1: {(1,
    (0,
     1300,
     0,
     1685)): {0: {(0,
      (0,
       71,
       0,
       527)): {0: {(3,
        (0,
         1,
         0,
         205)): {0: {(2,
          (0,
           1,
           0,
           47)): {0: {(4,
            (0,
             1,
             0,
             12)): {0: {(5, (0, 1, 0, 3)): {0: {'leaf': [1, [1]]},
              1: {'leaf': [3, [3]]}}}, 1: {'leaf': [3, [5]]}, 2: {'leaf': [3,
              [4]]}}}, 1: {'leaf': [3, [10]]}, 2: {'leaf': [3,
            [12]]}, 3: {'leaf': [3, [13]]}}}, 1: {'leaf': [3,
          [55]]}, 2: {'leaf': [3, [48]]}, 3: {'leaf': [3, [55]]}}}, 1: {(2,
        (0,
         1,
         0,
         193)): {0: {(4,
          (0,
           1,
           0,
           46)): {0: {(3,
            (0,
             1,
             0,
             9)): {0: {(5, (0, 1, 0, 1)): {0: {'leaf': [1, [1]]},
              1: {'leaf': [3, [1]]}}}, 1: {'leaf': [3, [1]]}

In [96]:
print(len(model._get_subtrees(model.main_tree)[-1]))

36


In [101]:
model.summary()
print(model.score(X_train, y_train))
print(model.score(X_val, y_val))
print(model.score(X_test, y_test))


Impurity: gini
Number of nodes: 1083
Number of leafs: 776
Max depth: 8
1.0
0.9872685185185185
0.9807098765432098


In [102]:
model.prune(X_val,y_val,max_prune_depth=2)
model.summary()
print(model.score(X_train, y_train))
print(model.score(X_val, y_val))
print(model.score(X_test, y_test))

hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
Impurity: gini
Number of nodes: 583
Number of leafs: 411
Max depth: 6
0.9759700176366843
0.9710648148148148
0.9629629629629629
